In [ ]:
# Import and install necessary modules
!pip install igraph
!pip install cairocffi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import igraph as ig
import cairocffi
from statistics import mode
from collections import Counter
from math import floor, ceil
from heapq import nlargest
from random import choices, choice
import seaborn as sns
import time
import warnings

In [ ]:
class Election:

    curr = True                       # The current composition of the graph (True as a placeholder)
    ta_progress = []                  # Keep track of the votes over time_advance iterations
    smp_seats, prop_seats = [], []    # No. seats won in proportional and smp
    votes = []                        # Final amount of votes that Party 1 has
    rand_change = 0                   # No. random changes over no_elections elections
    idx_smp, idx_prop = [], []        # Keeping track of indexes over elections
    graph_degrees = []                # Min, mean, max degree
    graph_connected = []              # Checking whether the graph is connected or not


    def __init__(self, n = 500, no_parties = 3, t = 0, iters = 5000, no_elecs = 500,
                 no_seats = 10, alpha = 0.05, p = 0.8, q = 0.1805, zeal = 0):

        self.n = n                        # Number of nodes (voters)
        self.no_parties = no_parties      # Number of political parties (or 'States') available to vote for
        self.iters = iters                # Max. number of changes possible for the graph (not every iteration results in a change)
        self.t = t                        # Time (starts at 0 when the object is created, +1 each election)
        self.no_elecs = no_elecs          # Number of elections to perform
        self.alpha = alpha                # Probability of random State reassignment after neighbour interaction
        self.no_seats = no_seats          # Number of seats available in a system
        self.p = p                        # Diagonal entries for stochastic block model preference matrix in init_graph
        self.q = q                        # Other entries in preference
        self.zeal = zeal                  # Percentage chance (between 0 and 100) that any one person is a zealot


    # Inside this class there are the following functions:
      # init_graph() creates a graph and assigns a State to each node
      # time_advance() performs iterations of the voter model https://en.wikipedia.org/wiki/Voter_model between elections
      # time_adv_runner() runs time advance and plots the line graph showing development of party votes over time
      # smp performs() a plurality election with single member districts https://en.wikipedia.org/wiki/Plurality_voting
      # proportional() performs a proportional representation election with no entry threshold
      # elec_runner() runs each of init_graph, time_advance, smp and proportional for no_elecs runs
      # hist() plots a histogram of the results of no_elecs elections
      # plot_network() allows visualisation of a social network, either for state or interaction
      # elec_stats() returns averaged political indexes over no_elecs elections

    def init_graph(self):
        # init_graph initialises a graph with n nodes to begin the simulation

        if self.t == 0: self.graph_degrees, self.graph_connected = [], []
        a = self.no_seats

        # Raise an error if zeal isn't a probability
        if self.zeal > 1 or self.zeal < 0:
          raise ValueError(f"zeal should be between 0 and 1\n")

        # Stochastic Block Model
        # Define a preference matrix for intra- and inter-community connection probabilities
        pm = np.identity(a)*self.p + (np.ones((a, a)) - np.identity(a))*self.q

        # Get a list of block sizes (divides the list of nodes into self.no_seats seats)
        split = np.array_split(range(self.n), a)
        block_sizes = [len(i) for i in split]

        # Use this to define the graph
        start_graph = ig.Graph.SBM(n = self.n, pref_matrix = list(pm), block_sizes = block_sizes)

        # Assign a State to each node (there are no_parties States)
        start_graph.vs()["State"] = np.random.randint(0, self.no_parties, self.n)

        # Assign an Interaction (normal or zealot) to each node - zealots only allowed in Party 1
        if self.zeal == 0:
            start_graph.vs()["Interaction"] = "Normal"
        else:
          for i in start_graph.vs():
            if i["State"] == 0:
              i["Interaction"] = choices(["Normal", "Zealot"], weights = [round(100*(1 - self.zeal)), round(100*self.zeal)], k = 1)[0]
            else:
              i["Interaction"] = "Normal"

        start_graph.get_adjacency
        a = start_graph.degree()[:]
        self.graph_degrees.append([np.min(a), np.max(a), np.mean(a), np.sum(a)/2])
        self.graph_connected.append(start_graph.is_connected())
        self.curr = start_graph


    def time_advance(self):
        # time_advance performs the (noisy if p > 0) voter model for self.iters iterations

        self.init_graph()
        current_graph, a, n = self.curr, self.no_parties, self.n
        if self.t == 0: self.ta_progress, self.votes = [], []

        # Initial State of voters
        vote_counts = {}
        for k in current_graph.vs()["State"]:
            vote_counts[k] = vote_counts.get(k, 0) + 1
        vote_counts = dict(sorted(vote_counts.items()))

        # Preallocating the data for plotting
        if self.iters >= 1000: this_elec_plot = [[0]*a]*ceil(self.iters/100)
        else: this_elec_plot = [[0]*a]*self.iters

        # Removing dot notation in loop is slightly more efficient
        randint, rand = np.random.randint, np.random.random
        vert = current_graph.vs

        # Loop for iters time steps
        for iter in range(self.iters):

            # Select a random integer between all nodes
            i = randint(0, n)
            active = vert[i]

            # Get the number of neighbours, and how many there are (skip if 0)
            neigh_list = active.neighbors()

            # Select a random integer between 0 and neigh
            j = randint(0, len(neigh_list))

            # Match the active State to the neighbour State if not zealot
            if active["Interaction"] == "Normal": active["State"] = neigh_list[j]["State"]

            # Implement random change for node i if Interaction is normal
            k = rand()
            if k <= self.alpha and active["Interaction"] == "Normal":
              active["State"] = choice([i for i in range(0,self.no_parties) if i != active["State"]])
              self.rand_change += 1

            # For ensuring every party is listed even if they have 0 votes
            zerokeys, zerovalues = range(self.no_parties), [0]*self.no_parties
            zerodict = dict(zip(zerokeys, zerovalues))

            # Data for plotting
            vote_counts = {}
            for k in current_graph.vs()["State"]:
              vote_counts[k] = vote_counts.get(k, 0) + 1
            vote_counts = {**zerodict, **vote_counts}

            # Not including all of the data for plotting, unless self.iters is low enough
            if self.iters >= 1000:
              if iter % 100 == 0:
                this_elec_plot[int(iter/100)] = list(vote_counts.values())
            else:
              this_elec_plot[iter] = list(vote_counts.values())

        self.ta_progress.append(this_elec_plot)
        self.votes.append(this_elec_plot[-1])


    def time_adv_runner(self):
        # This function performs the voter model iterations, without performing an election
        # It can be used to show the evolution of voter states over self.iters iterations

        self.t == 0
        leg = [f'Party {str(i + 1)}' for i in range(self.no_parties)]
        for run in range(self.no_elecs):
          self.time_advance()
          self.t += 1

        # Plot the mean of the data collected from running time_advance
        plot_data = 100*np.mean(self.ta_progress, axis = 0)/self.n
        fig = plt.figure()
        for i in range(plot_data.shape[1]):
          plt.plot(plot_data[:, i], label = 'Party ')
        plt.xlabel("Iterations of the voter model (hundreds)")
        plt.ylabel("% votes for each party")
        plt.ylim([0, 100])

        # Save output file
        filename = f'tar_rand_{self.p}_zeal_{self.zeal}'
        plt.savefig(f'{filename}.png', bbox_inches = 'tight')
        plt.close(fig)


    def smp(self):
        # Function to simulate a single-member plurality election with single-member districts
        # init_graph and time_advance are run in a separate function

        if self.t == 0: self.smp_seats, self.smp_votes, self.smp_total_ties = [], [], 0

        current_graph, seat_winners, a = self.curr, [], range(self.no_parties)
        sw_ap = seat_winners.append

        split = np.array_split(range(self.n), self.no_seats)
        iterator = [0] + list(np.cumsum([len(i) for i in split]))

        # Divide vertices into the predefined constituencies
        for iter in range(len(iterator)-1):
            vertices = list(range(iterator[iter], iterator[iter+1]))
            seat_scores = [current_graph.vs[j]["State"] for j in vertices]

            # Count the votes in each seat
            vote_counts = {}
            for k in seat_scores:
              vote_counts[k] = vote_counts.get(k, 0) + 1
            vote_counts = dict(sorted(vote_counts.items()))
            max_count = max(vote_counts.values())

            # Check for a tie in a seat
            poss_dup = Counter(vote_counts.values())
            result = {k: v for k, v in vote_counts.items() if poss_dup[v] > 1 and v == max_count}

            # Pick a winner at random if there is a tie
            if len(result) > 0:
              win_idx = np.random.randint(0, len(result))
              party = list(result.keys())[win_idx]
              sw_ap(party)
              self.smp_total_ties += 1

            # If not, the winning party is simply the one with the most votes
            else:
              party = list(vote_counts.keys())[list(vote_counts.values()).index(max_count)]
              sw_ap(party)

        # Ensure every party is listed even if they have 0 votes
        zerokeys, zerovalues = a, [0]*self.no_parties
        zerodict, vote_totals, seat_totals = dict(zip(zerokeys, zerovalues)), {}, {}

        # Get the number of votes in total
        for i in current_graph.vs()["State"]:
          vote_totals[i] = vote_totals.get(i, 0) + 1

        # Get the number of seats won
        for i in seat_winners:
          seat_totals[i] = seat_totals.get(i, 0) + 1
        seat_totals = {**zerodict, **seat_totals}

        self.smp_seats.append(list(seat_totals.values()))


    def proportional(self):
      # Performs proportional representation with no entry threshold.
      # Percentage of votes = percentage of seats (rounded using Hare quota)

      if self.t == 0: self.prop_seats, self.prop_votes, self.prop_total_ties = [], [], 0
      current_graph, a = self.curr, range(self.no_parties)

      # Dictionaries for vote count, vote percentage, number of seats allocated, and remainder from Hare Quota seat allocation
      vote_counts, vote_perc, seat_totals, rem = {}, {}, {}, {}
      zerokeys, zerovalues = a, [0]*self.no_parties
      zerodict = dict(zip(zerokeys, zerovalues))

      # Total votes
      for i in current_graph.vs()["State"]:
        vote_counts[i] = vote_counts.get(i, 0) + 1
      vote_counts = dict(sorted(vote_counts.items()))
      vote_counts = {**zerodict, **vote_counts}

      # Vote percentage
      for key, value in enumerate(vote_counts.items()):
        vote_perc[key] = (value[1] / self.n)*100

      # Distribute seats using largest remainder method
      # Initially allocate ⌊qi⌋ seats
      threshold = (self.n/self.no_seats)

      for key, value in enumerate(vote_counts.items()):
        seat_totals[key] = floor(value[1]/threshold)
        rem[key] = value[1] % threshold

      # Order all parties by their decimal part of the quota
      remaining_seats = self.no_seats - sum(list(seat_totals.values()))
      largest_dec = nlargest(remaining_seats, list(rem.values()))

      for i in largest_dec:
        # Find the key for where the largest remainders are
        k = list(rem.keys())[list(rem.values()).index(i)]
        # Add 1 to each seat count which features in largest_dec
        seat_totals[k] = seat_totals.get(k, 0) + 1

      # Ensure every party is listed even if they have 0 votes
      seat_totals = {**zerodict, **seat_totals}

      self.prop_seats.append(list(seat_totals.values()))


    def elec_runner(self):
    # This function creates a new network for each election in self.no_elecs, performs
    # time_advance then holds SMP and PR elections on the same graph
      for i in range(self.no_elecs):
        self.init_graph()
        self.time_advance()
        self.smp()
        self.proportional()
        self.t += 1


    def hist(self, title = False):
      # This function runs elec_runner then plots a histogram of seat distribution probability

      # Reset time and run elec_runner()
      self.t = 0
      self.elec_runner()

      for i in range(self.no_parties):

        # Get vote and seat percentages
        votes = 100*np.array(self.votes)[:, i]/self.n
        smp = 100*np.array(self.smp_seats)[:, i]/self.no_seats
        prop =  100*np.array(self.prop_seats)[:, i]/self.no_seats

        # Mean and standard deviation
        vav, vstd = np.mean(votes), np.std(votes)
        sav, sstd = np.mean(smp), np.std(smp)
        pav, pstd = np.mean(prop), np.std(prop)

        # Creating a probability histograph with 10 bins
        bin_lims = np.linspace(0, 100, 11)
        bin_centers = 0.5*(bin_lims[:-1]+bin_lims[1:])
        bin_widths = bin_lims[1:]-bin_lims[:-1]
        v_hist, _ = np.histogram(votes, bins = bin_lims)
        s_hist, _ = np.histogram(smp, bins = bin_lims)
        p_hist, _ = np.histogram(prop, bins = bin_lims)

        # Normalising
        shb, phb, vhb = s_hist/self.no_elecs, p_hist/self.no_elecs, v_hist/self.n

        # Plot histograms for seats, means +/- 1 standard deviation for both systems on the same graph
        fig1, ax1 = plt.subplots()
        ax1.bar(bin_centers, shb, width = bin_widths, align = 'center', color = 'blue', alpha = 0.5)
        ax1.bar(bin_centers, phb, width = bin_widths, align = 'center', color = 'orange', alpha = 0.5)
        ax1.vlines([sav, pav], ymin = 0, ymax = 1, color = ['blue', 'orange'], linewidth = 2)
        ax1.vlines([sav + sstd, sav - sstd], ymin = 0, ymax = 1, color = 'blue', linestyle = '--')
        ax1.vlines([pav + pstd, pav - pstd], ymin = 0, ymax = 1, color = 'orange', linestyle = '--')
        if title == True:
          ax1.set_title(f'Seats won by Party {i+1} with {int(100*self.zeal)}% zealot probability')
        elif title == "Custom":
          title_is = input("Choose graph title: ")
          ax1.set_title(f'{title_is}')
        ax1.set_xlabel(f'% seats won by Party {i+1}')
        ax1.set_ylabel('Probability')
        ax1.set_ylim([0, 1])

        filename = input("Enter file name: ")
        plt.savefig(f'{filename}.png', bbox_inches = 'tight')
        plt.close(fig1)

        # Histogram for votes
        fig3, ax3 = plt.subplots()
        ax3.bar(bin_centers, vhb, width = bin_widths, align = 'center', color = 'green', alpha = 0.8)
        ax3.vlines(vav, ymin = 0, ymax = 1, color = 'green', linewidth = 2)
        ax3.vlines([vav + vstd, vav - vstd], ymin = 0, ymax = 1, color = 'green', linestyle = '--')
        if title == True:
          ax3.set_title(f'Votes won by Party {i+1} with {int(100*self.zeal)}% Zealots')
        elif title == "Custom":
          title_is = input("Choose graph title: ")
        ax3.set_xlabel(f'% votes won by Party {i+1}')
        ax3.set_ylabel('Probability')
        ax3.set_ylim([0, 1])

        filename = input("Enter file name: ")
        plt.savefig(f'{filename}.png', bbox_inches = 'tight')
        plt.close(fig3)



    def plot_network(self, plot_mode = "Interaction"):

        # Create a graph (if there isn't already an existing one)
        if self.t == 0: self.init_graph()
        current_graph = self.curr

        # Decide whch features we want the graph to show - Interaction, or political opinion
        if plot_mode == "Interaction":
          colour_dict = {"Zealot": "blue", "Normal": "red"}
          current_graph.vs["color"] = [colour_dict[inter] for inter in current_graph.vs["Interaction"]]
        elif plot_mode == "State":
          cd_keys, cd_values = range(self.no_parties), sns.color_palette(None, 3)
          colour_dict = dict(zip(cd_keys, cd_values))
          current_graph.vs["color"] = [colour_dict[State] for State in current_graph.vs["State"]]

        # Plot the graph
        fig, ax = plt.subplots()
        ig.plot(current_graph, target = ax, bbox = (600, 600))
        filename = input("Enter file name: ")
        plt.savefig(f'{filename}.png', bbox_inches = 'tight')
        plt.close(fig)


    def elec_stats(self):
        # This function performs statistics and calculates indexes based on the results from elec_runner
        # It should be done at the end of the number of elections performed

        self.idx_smp, self.idx_prop = [], []

        # smp and proportional vote and seat percentages
        v_pc = 100*np.array(self.votes)/self.n
        smp_s_pc = 100*np.array(self.smp_seats)/self.no_seats
        prop_s_pc = 100*np.array(self.prop_seats)/self.no_seats

        # Calculating the mean and STD seats won
        mean_v, std_v = np.mean(v_pc, axis = 0), np.std(v_pc, axis = 0)
        mean_s_smp, mean_s_prop = np.mean(smp_s_pc, axis = 0), np.mean(prop_s_pc, axis = 0)
        smp_s_std, prop_s_std = np.std(smp_s_pc, axis = 0), np.std(prop_s_pc, axis = 0)

        # Calculating the Gallagher index
        gall_smp = np.mean(np.sqrt(0.5*np.sum(np.square(v_pc - smp_s_pc), axis = 1)))
        gall_prop = np.mean(np.sqrt(0.5*np.sum(np.square(v_pc - prop_s_pc), axis = 1)))

        # Calculating the Loosemore-Hanby index
        lh_smp = np.mean(0.5*np.sum(np.abs(v_pc - smp_s_pc), axis = 1))
        lh_prop = np.mean(0.5*np.sum(np.abs(v_pc - prop_s_pc), axis = 1))

        # Calculating effective number of parties (Laakso and Taagepara) - for seats
        enp_smp_s = np.mean(1/np.sum(np.square(smp_s_pc/100), axis = 1))
        enp_prop_s = np.mean(1/np.sum(np.square(prop_s_pc/100), axis = 1))

        # ENP for votes (just one since it's the same graph)
        enp_v = np.mean(1/np.sum(np.square(v_pc/100), axis = 1))

        results_smp = [mean_v, std_v, mean_s_smp, smp_s_std, gall_smp, lh_smp, enp_smp_s, enp_v]
        results_prop = [mean_v, std_v, mean_s_prop, prop_s_std, gall_prop, lh_prop, enp_prop_s, enp_v]

        self.idx_smp.append(results_smp)
        self.idx_prop.append(results_prop)